## 3.1 Data transferred and ingested in butler and access to data

Get the same ras as we did on the summit

In [3]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry

repo = '/repo/embargo'
# raw all colleciton contains all images 
collections=['LATISS/raw/all']
butler = Butler(repo)
cols = butler.registry.queryCollections("*LATISS*raw*")
for c in cols :
    print(c)


butler = Butler(repo, collections=collections)
registry = butler.registry



LATISS/raw/all


In [5]:
dataId={'day_obs':20230509}

dt='raw'
where="exposure.day_obs = 20230509"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])



There are 572 raw in collection: ['LATISS/raw/all'] in: /repo/embargo for exposure.day_obs = 20230509
0 2023050900001
0 2023050900572


### latency ..

In [ ]:
for ref in dataRefs:
    print(ref.dataId['

In [ ]:
print(calexp)

That was a bit specific we can get a broader set ...

Now we have 1 image
lets look at this one calexp .. you do not need this following cell it is just in case you want to chekc the image.